In [ ]:
!pip install transformers
!pip install evaluate
!pip install optimum

In [ ]:
import transformers
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import evaluate
from datasets import load_dataset
from optimum.pipelines import pipeline

In [ ]:
model_id = "juliensimon/autotrain-food101-1471154050"
task_type = "image-classification"

In [ ]:
model = AutoModelForImageClassification.from_pretrained(model_id)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)
pipe = transformers.pipeline(task_type, model=model, feature_extractor=feature_extractor)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
dataset_id = "food101"
data = load_dataset(dataset_id, split="validation[:25%]", num_proc=8)
print(data)
metric = evaluate.load("accuracy")
evaluator = evaluate.evaluator(task_type)

Setting num_proc from 8 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Setting num_proc from 8 back to 1 for the validation split to disable multiprocessing as it only contains one shard.


Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 6312
})


In [ ]:
data

Dataset({
    features: ['image', 'label'],
    num_rows: 6312
})

In [ ]:
def evaluate_pipeline(pipeline):
    results = evaluator.compute(
        model_or_pipeline=pipeline,
        data=data,
        metric=metric,
        label_column="label",
        label_mapping=model.config.label2id,
    )
    return results

In [ ]:
print("*** Original model")
classifier = transformers.pipeline(task_type, model_id, device=0)
results = evaluate_pipeline(classifier)
print(results)

*** Original model


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


{'accuracy': 0.8591571609632446, 'total_time_in_seconds': 145.02861557300002, 'samples_per_second': 43.52244538129001, 'latency_in_seconds': 0.02297665012246515}


In [ ]:
print("*** BetterTransformer model")

classifier_bt = pipeline(task_type, model_id, accelerator="bettertransformer", device=0)
results = evaluate_pipeline(classifier_bt)
print(results)


*** BetterTransformer model


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


{'accuracy': 0.8588403041825095, 'total_time_in_seconds': 137.2407309419998, 'samples_per_second': 45.99217707946743, 'latency_in_seconds': 0.021742828096007572}
